<a href="https://colab.research.google.com/github/Reaper-ai/ML_AI/blob/main/01_Simple_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
print("setup complete")

setup complete


In [ ]:
print("Is CUDA available? ", torch.cuda.is_available())
print("Current device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


Is CUDA available?  True
Current device: cuda


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load Data

data_train = datasets.FashionMNIST(root="data",
                                   train=True,
                                   download=True,
                                   transform=transforms.ToTensor())
data_test = datasets.FashionMNIST(root="data",
                                  train=False,
                                  download=True,
                                  transform=transforms.ToTensor())


In [ ]:
batch_size = 64

train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(data_test, batch_size=batch_size, shuffle=True)

In [ ]:
# Create Model

class SimpleNN(nn.Module):
  def __init__(self, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.flatten = nn.Flatten()
    self.layers = nn.Sequential(
          nn.Linear(28*28, 128),
          nn.ReLU(),
          nn.Linear(128, 128),
          nn.ReLU(),
          nn.Linear(128, 10),
          )


  def forward(self, x):
    x = self.flatten(x)
    logits = self.layers(x)

    return logits

my_first_model = SimpleNN()
my_first_model.to(device)

SimpleNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(my_first_model.parameters(), lr=0.01)


In [ ]:
def train_loop(model, train_loader, loss_fn, optimizer):

  model.train()

  for batch, (X,y) in enumerate(train_loader):

    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
def test_loop(model, test_loader, loss_fn):
  model.eval()
  size = len(test_loader.dataset)
  num_batches = len(test_loader)
  test_loss, correct = 0, 0

  with torch.inference_mode():
    for batch, (X,y) in enumerate(test_loader):

      X, y = X.to(device), y.to(device)
      test_pred = model(X)
      test_loss += loss_fn(test_pred, y)
      correct += (test_pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(my_first_model,train_loader, loss_fn, optimizer)
    test_loop(my_first_model, test_loader,loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.440391 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 83.5%, Avg loss: 0.454663 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 84.7%, Avg loss: 0.438633 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 84.3%, Avg loss: 0.434936 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 85.4%, Avg loss: 0.414663 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.440204 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 85.3%, Avg loss: 0.412763 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.422248 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 85.3%, Avg loss: 0.421405 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 85.9%, Avg loss: 0.401257 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 83.9%, Avg los